In [21]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import pandas as pd
from datetime import datetime
import time

# Initialize the WebDriver
driver = webdriver.Chrome()
driver.get("https://keeptradecut.com/dynasty-rankings") 
wait = WebDriverWait(driver, 20)

# Build a blank list to catch table scrape.
data = []

# KTC prompts a popup window to grade players, need to close it dynamically to scrape the underlying table.
try:
    pop_up = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "ktc-dont-know")))
    dismiss_button = pop_up.find_element(By.ID, "dont-know")
    time.sleep(2)
    dismiss_button.click()
    wait.until(EC.invisibility_of_element((By.CLASS_NAME, "ktc-dont-know")))
    print("Pop-up dismissed.")
except TimeoutException:
    print("KTC Ranking Popup Failed to Close")

# Do-While loop to scrape table, paginate to next 50 rows, append to existing scrape.
while True: 
    try:
        # Scrape the data on the current page
        players = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "onePlayer")))
        
        for player in players:
            try:
                rank = player.find_element(By.CLASS_NAME, "rank-number").text
                name = player.find_element(By.CSS_SELECTOR, ".player-name a").text
                try:
                    team = player.find_element(By.CLASS_NAME, "player-team").text
                except NoSuchElementException:
                    team = None
                position = player.find_element(By.CLASS_NAME, "position").text
                age_elements = player.find_elements(By.CSS_SELECTOR, ".position-team p")
                age = age_elements[1].text if len(age_elements) > 1 else None 
                tier = player.find_element(By.CSS_SELECTOR, ".player-tier .position").text
                value = player.find_element(By.CLASS_NAME, "value").text

                data.append({
                    "Rank": rank,
                    "Player": name,
                    "Team": team,
                    "Position": position,
                    "Age": age,
                    "Tier": tier,
                    "Value": value
                })

            except (TimeoutException, NoSuchElementException):
                print("Error scraping player data")
                continue

        next_button = driver.find_element(By.CSS_SELECTOR, ".pagination-arrow.arrow-right")
        if "inactive" in next_button.get_attribute("class"):
            print("Reached the final page. Stopping pagination.")
            break 
        driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
        time.sleep(1)
        next_button.click()
        print("Navigated to the next page.")
        time.sleep(2)

    except TimeoutException:
        print("Pagination button not found. Stopping pagination.")
        break  

# Convert list to a PD dataframe to prep for export.
df = pd.DataFrame(data)
df['Load_Date'] = datetime.now()
driver.quit()

Pop-up dismissed.
Navigated to the next page.
Navigated to the next page.
Navigated to the next page.
Navigated to the next page.
Navigated to the next page.
Navigated to the next page.
Navigated to the next page.
Navigated to the next page.
Navigated to the next page.
Reached the final page. Stopping pagination.
    Rank            Player Team Position        Age     Tier Value  \
0      1     Lamar Jackson  BAL      QB1  27.8 y.o.   Tier 1  9995   
1      2    Jayden Daniels  WAS      QB2  23.9 y.o.   Tier 1  9994   
2      3  Justin Jefferson  MIN      WR1  25.4 y.o.   Tier 1  9993   
3      4        Josh Allen  BUF      QB3  28.5 y.o.   Tier 1  9966   
4      5     Ja'Marr Chase  CIN      WR2  24.7 y.o.   Tier 2  9554   
..   ...               ...  ...      ...        ...      ...   ...   
495  496      Carson Wentz  KCC     QB77  31.9 y.o.  Tier 27   193   
496  497     Joshua Kelley  TEN    RB118    27 y.o.  Tier 27   189   
497  498      Kevin Harris   FA    RB119    24 y.o.  Ti